In [1]:
import pandas as pd

# Загрузка датасета
file_path = r"C:/Users/10509/Downloads/dataset.csv"  # Укажите путь к вашему файлу
data = pd.read_csv(file_path)

# Проверка структуры данных
print(data.head())

# Проверка на пропущенные значения
print(data.isnull().sum())


   Unnamed: 0  transaction_id             ip  device_id device_type  \
0           0         1157921    91.107.0.90      982.0   prtbl trm   
1           1         1595205   37.0.127.150     1033.0    port_trm   
2           2         1656884  185.233.0.240       96.0         atm   
3           3         1673909    91.107.0.90      982.0   prtbl trm   
4           4         1689203   188.43.0.135     1413.0         ATM   

   tran_code   mcc  client_id card_type  pin_inc_count card_status  \
0         30  7103  969838140     DEBIT              0      active   
1         30  2031  969838140     DEBIT              0      active   
2         25  4629  969838140     DEBIT              0      active   
3         30  4629  969838140     DEBIT              0      active   
4         10  5840  969838140     DEBIT              0      active   

  expiration_date             datetime     sum   oper_type    balance  
0      2030-06-10  2022-03-09 09:56:51  510.22    transfer  670814.78  
1      2

In [2]:
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
# Удаляем столбец 'Unnamed: 0'
data = data.drop(columns=['Unnamed: 0'])

# Сортировка данных по 'client_id' datetime
data_sorted = data.sort_values(by=['client_id', 'datetime'], ascending=[True, True]).reset_index(drop=True)


# Печать отсортированного датасета
print(data_sorted)

       transaction_id             ip  device_id device_type  tran_code   mcc  \
0             1107483  109.126.0.150      376.0         atm         10  9534   
1             1132931    151.0.38.90      394.0         ATM        451  6735   
2             1297102  194.0.144.180      175.0     pos trm        451  7349   
3             1338602    195.58.0.15     1425.0         atm         10  9618   
4             1338948    188.0.0.105     1213.0    cash_out         20  2440   
...               ...            ...        ...         ...        ...   ...   
53074         9033061   86.111.0.225     1359.0    cash_out         20  5934   
53075         9153745  194.0.162.210      757.0         ATM         -1  5755   
53076         9300162   85.140.0.210     1267.0    cash_out         -1  5039   
53077         9964963   37.0.123.180      333.0     cash_in        451  4057   
53078         9964964   31.204.0.225      932.0         atm        451  4057   

       client_id card_type  pin_inc_cou

In [3]:
import geoip2.database

# Путь к базе данных GeoLite2
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

def get_city_from_ip(ip):
    try:
        # Получаем данные о местоположении
        response = reader.city(ip)
        city = response.city.name  # Извлекаем название города
        return city
    except geoip2.errors.AddressNotFoundError:
        return 'Unknown'  # Если IP-адрес не найден

# Пример использования
data_sorted['ip_city'] = data_sorted['ip'].apply(get_city_from_ip)
# Печать отсортированного датасета
print(data_sorted)

InvalidDatabaseError: Error opening database file (GeoLite2-City.mmdb). Is this a valid MaxMind DB file?

In [4]:
import os
print(os.getcwd())

C:\Windows\System32


In [9]:
pip install geoip2


Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# Пример подготовки данных
# Преобразуем 'datetime' в datetime, если это еще не сделано
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')

# Добавление временных признаков
data['day_of_week'] = data['datetime'].dt.dayofweek  # День недели
data['hour_of_day'] = data['datetime'].dt.hour  # Час дня
data['month_of_year'] = data['datetime'].dt.month  # Месяц

# Применим One-Hot Encoding к категориальным признакам
categorical_columns = ['device_type', 'oper_type', 'card_type']  # Список категориальных признаков
data_encoded = pd.get_dummies(data, columns=categorical_columns)

# Масштабируем данные после кодировки
features = ['sum', 'balance', 'pin_inc_count', 'day_of_week', 'hour_of_day', 'month_of_year'] + \
           [col for col in data_encoded.columns if col.startswith(('oper_type', 'device_type', 'card_type'))]

X = data_encoded[features]  # Признаки

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Упрощенная модель автоэнкодера
autoencoder = Sequential([
    Dense(64, activation='relu', input_dim=X_scaled.shape[1]),  # Входной слой с меньшими нейронами
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(X_scaled.shape[1], activation='linear')  # Выходной слой
])

# Компиляция и обучение автоэнкодера с использованием EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=32, shuffle=True, verbose=1, callbacks=[early_stopping])

# Получение восстановленных данных (предсказания)
reconstructed = autoencoder.predict(X_scaled)

# Вычисление ошибки восстановления
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=1)

# Определение порога для аномалий (например, верхние 5% ошибок считаются аномалиями)
threshold = np.percentile(mse, 95)  # Верхние 5% считаются аномалиями
data['anomaly'] = mse > threshold  # 1 = аномалия, 0 = нормальная операция

# Проверим, что столбец 'anomaly' был создан
print(data[['transaction_id', 'anomaly']].head())

# Разделение данных на обучающую и тестовую выборки
y = data['anomaly'].map({True: 1, False: 0})  # Преобразуем метки аномалии в 1 и 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Масштабируем данные
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение модели Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_scaled, y_train)

# Предсказания на тестовой выборке
y_pred = logreg.predict(X_test_scaled)

# Оценка результатов
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"AUC: {auc}")


Epoch 1/50


E:\anaconda\envs\rinhak\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1659/1659 ━━━━━━━━━━━━━━━━━━━━ 2s 576us/step - loss: 0.4063
Epoch 2/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 533us/step - loss: 0.1132
Epoch 3/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 540us/step - loss: 0.0407
Epoch 4/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 542us/step - loss: 0.0355
Epoch 5/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 575us/step - loss: 0.0360
Epoch 6/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 608us/step - loss: 0.0099
Epoch 7/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 593us/step - loss: 0.0106
Epoch 8/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 577us/step - loss: 0.0141
Epoch 9/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 590us/step - loss: 0.0071
Epoch 10/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 568us/step - loss: 0.0135
Epoch 11/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 561us/step - loss: 0.0314
Epoch 12/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 544us/step - loss: 0.0286
Epoch 13/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 570us/step - loss: 0.0262
Epoch 14/50
1659/1659 ━━━━━━━━━━━━━━━━━━━━ 1s 560us/step - loss: 0.0251
Epoch 15/50


In [5]:

data.dtypes


Unnamed: 0                  int64
transaction_id              int64
ip                         object
device_id                 float64
device_type                object
tran_code                   int64
mcc                         int64
client_id                   int64
card_type                  object
pin_inc_count               int64
card_status                object
expiration_date            object
datetime           datetime64[ns]
sum                       float64
oper_type                  object
balance                   float64
day_of_week                 int32
hour_of_day                 int32
month_of_year               int32
anomaly                      bool
dtype: object

In [8]:
transaction_count = data.groupby('client_id').size().reset_index(name='transaction_count')

# Печать результатов
print(transaction_count)

     client_id  transaction_count
0    101648380                 51
1    101814879                 59
2    102589938                 58
3    105878057                 52
4    106476302                 55
..         ...                ...
995  991618345                 58
996  992042996                 52
997  997093996                 51
998  998717557                 54
999  999660534                 59

[1000 rows x 2 columns]


In [11]:
data_sorted = data.sort_values(by='client_id', ascending=True).reset_index(drop=True)

# Печать отсортированного датасета
print(data_sorted)

       Unnamed: 0  transaction_id            ip  device_id device_type  \
0            6368         4359399   151.0.38.90      394.0         ATM   
1            6370         4807117  213.151.0.30      998.0         ATM   
2            6397         9767466   193.31.0.90      594.0         ATM   
3            6398         9910432  80.234.0.105      418.0   prtbl trm   
4            6400         9971565  80.234.0.105      418.0   prtbl trm   
...           ...             ...           ...        ...         ...   
53074       36296         4212845  151.0.38.120      398.0         ATM   
53075       36282         1833016   95.106.0.45      547.0    port_trm   
53076       36283         2047408    46.173.0.0      487.0   prtbl trm   
53077       36284         2067293  85.140.0.210     1267.0    cash_out   
53078       36278         1509944  193.0.151.90       58.0    cash_out   

       tran_code   mcc  client_id card_type  pin_inc_count card_status  \
0             10  6776  101648380    

In [12]:
# Удаляем столбец 'Unnamed: 0'
data = data.drop(columns=['Unnamed: 0'])

# Сортировка данных по 'client_id'
data_sorted = data.sort_values(by='client_id', ascending=True).reset_index(drop=True)

# Печать отсортированного датасета
print(data_sorted)

       transaction_id            ip  device_id device_type  tran_code   mcc  \
0             4359399   151.0.38.90      394.0         ATM         10  6776   
1             4807117  213.151.0.30      998.0         ATM         20  6964   
2             9767466   193.31.0.90      594.0         ATM         30  8731   
3             9910432  80.234.0.105      418.0   prtbl trm         30  6558   
4             9971565  80.234.0.105      418.0   prtbl trm        451  2042   
...               ...           ...        ...         ...        ...   ...   
53074         4212845  151.0.38.120      398.0         ATM         25  8326   
53075         1833016   95.106.0.45      547.0    port_trm         -1  4057   
53076         2047408    46.173.0.0      487.0   prtbl trm        451  8785   
53077         2067293  85.140.0.210     1267.0    cash_out         25  4057   
53078         1509944  193.0.151.90       58.0    cash_out         20  6748   

       client_id card_type  pin_inc_count card_stat